In [ ]:
import scipy.stats as stats
import numpy as np
import scipy.optimize as optimize
import math
import matplotlib.pyplot as plt
import pandas as pd
import itertools
from itertools import combinations_with_replacement
from itertools import combinations
from itertools import permutations

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

criteria_dict = {"cs_name_val" : 10}
project = signac.get_project()
save_csv = False
save_figs = False
analyzer = General_Analysis(criteria_dict, project, save_csv)
plotters = Plotters(analyzer, save_figs)


def grid_sampling(num_points, bounds):
        """
        Generates Grid sampled data
        
        Parameters
        ----------
        num_points: int, number of points in LHS, should be greater than # of dimensions
        bounds: ndarray, array containing upper and lower bounds of elements in LHS sample. Defaults of 0 and 1
        
        Returns:
        ----------
        grid_data: ndarray, (num_points)**bounds.shape[1] grid sample of data
        
        """
        #Generate mesh_grid data for theta_set in 2D
        #Define linspace for theta
        params = np.linspace(0,1, num_points)
        #Define dimensions of parameter
        dimensions = bounds.shape[1]
        #Generate the equivalent of all meshgrid points
        df = pd.DataFrame(list(itertools.product(params, repeat=dimensions)))
        df2 = df.drop_duplicates()
        scaled_data = df2.to_numpy()
        #Normalize to bounds 
        lower_bound = bounds[0]
        upper_bound = bounds[1]
        grid_data = scaled_data*(upper_bound - lower_bound) + lower_bound 
        return grid_data

##New Cell

## define function that includes nonlinear model
def model(theta_guess, theta_ref, x, indeces_to_consider):
    '''
        """
    Creates Muller potential values given a guess for "a"
    Parameters
    ----------
        a_guess: ndarray, guess value for a
        Constants: ndarray, The array containing the true values of Muller constants
        x: ndarray, Independent variable data (exp or pred)
    Returns
    -------
        y_model: ndarray, The simulated Muller potential given the guess
    '''
    #Define an array to store y values in
    y_data = []
    #Loop over all theta values
    for i in range(len(x)):
        #Create model coefficient from true space substituting in the values of param_space at the correct indeces
        model_coefficients = theta_ref.copy()
        #Replace coefficients a specified indeces with their theta_val counterparts
        model_coefficients[indeces_to_consider] = theta_guess              
        #Create y data coefficients
        y_data.append(calc_cs8_10_polynomial(model_coefficients, x[i]))

    #Convert list to array and flatten array
    y_model = np.array(y_data).flatten()
    
    return y_model

# print(model(a_guess, theta_ref, Xexp, min_muller, indeces_to_consider))

##New Cell

# Create a function to optimize, in this case, least squares fitting
def regression_func(theta_guess, theta_ref, x, indeces_to_consider, y):
    '''
    Function to define regression function for least-squares fitting
    Arguments:
        a_guess: ndarray, guess value for a
        Constants: ndarray, The array containing the true values of Muller constants
        x: ndarray, experimental X data (Inependent Variable)
        y: ndarray, experimental Y data (Dependent Variable)
    Returns:
        e: residual vector
    '''
    
    error = y.flatten() - model(theta_guess, theta_ref, x, indeces_to_consider).flatten(); #NOTE: Least squares will calculate sse based off this to minimize
    
    return error

# print(regression_func(a_guess, Constants, Xexp, Yexp))
def sse_func(theta_guesses, theta_ref, indeces_to_consider, Xexp, Yexp):
    '''
    Function to define define sum of squared error function for heat map
    Arguments:
        xx: An N X D array of all a_1 values
        yy: An D X N array of all a_2 values
        x: independent variable vector (predicted x values including noise)
        y: dependent variable vector (predicted y values on Heat Map)
    Returns:
        sse: N x N sum of squared error matrix of all generated combination of xx and yy
    '''
    #Initialize sse grid
    sse = np.zeros(len(theta_guesses))
    
    #For each guess
    for i in range(len(theta_guesses)):
        #Evaluate the model
        y_sim = model(theta_guesses[i], theta_ref, Xexp, indeces_to_consider)
        #Calculate SSE
        sse[i] = np.sum((y_sim - Yexp)**2)
     
    sse = sse.reshape(int(np.sqrt(len(theta_guesses))), -1).T
    
    return sse


In [ ]:
%%time
# Create synthetic data assuming the following values for theta

param_name_str = "t1t2t3t4t5"
indeces_to_consider = set_idcs_to_consider(8, param_name_str)

theta_names = ['theta_1', 'theta_2', 'theta_3', 'theta_4', 'theta_5']
bounds_x_l = [-5, -5]
bounds_x_u = [ 5,  5]
bounds_x = np.array([bounds_x_l, bounds_x_u])
bounds_theta_l = [-5,-5,-1,5, -5]
bounds_theta_u = [ 5, 5, 1, 10,  5]
theta_ref = np.array([1, -2, 0.5, 7, -3])  
theta_true = np.array([theta_ref[i] for i in indeces_to_consider] )
theta_true_names = np.array([theta_names[i] for i in indeces_to_consider] )

# This generates experimental data points
num_points = 5
Xexp = grid_sampling(num_points, bounds_x)
# Evaluate model and add noise based on assumed theta values
Yexp_org = np.array([ calc_cs8_10_polynomial(theta_ref, Xexp[i])  for i in range(len(Xexp)) ]).flatten()
std = np.average(Yexp_org)*0.05
noise = np.random.normal(size=len(Yexp_org), loc = 0, scale = std)
Yexp = Yexp_org + noise

In [ ]:
##New Cell

# Evaluate model based on the assumed experimental values
#Create Meshgrid for X1 and X2 and evaluate Y
len_mesh_data = 20
x1 = np.linspace(bounds_x_l[0],bounds_x_u[0],len_mesh_data)
x2 = np.linspace(bounds_x_l[1],bounds_x_u[1],len_mesh_data)
X1, X2 = np.meshgrid(x1,x2)
X_mesh = np.meshgrid(x1,x2)
X_mesh_reshape = np.array(X_mesh).reshape(2,-1).T
# #Creates an array for Y that will be filled with the for loop

#Calculate y_sim
Y =  [ calc_cs8_10_polynomial(theta_ref, X_mesh_reshape[i]) for i in range(len(X_mesh_reshape)) ] 
        
#Reshape to correct dimension (Is there an easier wat to do this?)        
param_names = ["X1", "X2"]
levels = [100]
idcs_to_plot = [0]
zz = np.array(Y).reshape(len_mesh_data,-1).T
z = np.array([zz])
z_titles = ["Polynomial"]
xbins = 5
ybins = 5
zbins = 900
title= None

param_info_dict = {"true":None, "min_sse":None, "names":param_names, "idcs":idcs_to_plot}
plotters.plot_nlr_heat_maps(X_mesh, z, z_titles, levels, param_info_dict, log_data = False)

In [ ]:
%%time

#Set seed and repeats
seed = 1
np.random.seed(seed)
repeats = 5

## specify bounds
lower = np.array([bounds_theta_l[i] for i in indeces_to_consider] )
upper = np.array([bounds_theta_u[i] for i in indeces_to_consider] )
bounds = (lower, upper) 

## specify initial guesses
theta_guess = np.random.uniform(low=lower, high=upper, size=(repeats, len(lower)) )
theta_vals = np.zeros((repeats, len(lower)))
l2_norms = np.zeros(repeats)
costs = np.zeros(repeats)
fxn_evals = np.zeros(repeats)

for i in range(repeats):
    Solution = optimize.least_squares(regression_func, theta_guess[i] ,bounds=bounds, method='trf',
                                      args=(theta_ref, Xexp, indeces_to_consider, Yexp),verbose=2)

    theta_min_obj = Solution.x
    costs[i] = Solution.cost
    theta_vals[i] = theta_min_obj
    #Note counting Jacobian and function evalauations as function evaluations
    fxn_evals[i] = Solution.nfev + Solution.njev
    
    del_theta = theta_min_obj - theta_true
    theta_L2_norm = np.linalg.norm(del_theta, ord = 2)
    l2_norms[i] = theta_L2_norm
    
nlr_theta = theta_vals[np.argmin(costs)]
nlr_l2_norm = l2_norms[np.argmin(costs)]
nlr_evals = fxn_evals[np.argmin(costs)]
print("Best Theta = ", nlr_theta)
print("Best Theta L2 norm = ", nlr_l2_norm)
print("Evaluations = ", nlr_evals)
print("theta_ref", theta_true)
Y_nlr_exp = model(nlr_theta, theta_ref, Xexp, indeces_to_consider)
error = (Yexp - Y_nlr_exp)
print("SSE = ", np.sum(error**2))

In [21]:
sigre = std**2
MSE = (error.T @ error)/(len(error) - 2)
Hess = Solution.jac.T @ Solution.jac
Covar = sigre * np.linalg.inv(Hess)
FIM = (1/sigre)*Hess
eigvals, eigvecs = np.linalg.eig(FIM)
k = np.max(eigvals)/np.min(eigvals)
print("MSE = ", MSE)
print("Experimental Variance = ", sigre)
print("Parameter Prediction Standard Deviation: \n", np.sqrt(np.diag(Covar)))
print("Covariance matrix:\n",Covar)
print("Det(FIM) = ", np.linalg.det(FIM))
print("Eigen Values (FIM):\n", eigvals)
print("Eigen Vectors (FIM)\n", eigvecs)
print("Condition Number (FIM): ", k)
print("Degree of precision loss (log10(k) of FIM): ", math.log10(k))

MSE =  5.0012829377571535
Experimental Variance =  6.25
Parameter Prediction Standard Deviation: 
 [0.14142136 0.14142135 0.04       0.03793618 0.03793618]
Covariance matrix:
 [[ 2.00000000e-02  3.88959123e-11  1.36160280e-19  4.39905678e-20
  -1.68794411e-19]
 [ 3.88959123e-11  1.99999996e-02  7.00126413e-11  5.33018054e-11
  -8.28338878e-11]
 [ 1.36160280e-19  7.00126413e-11  1.60000000e-03  1.91671634e-19
  -2.72416278e-19]
 [ 4.39905678e-20  5.33018054e-11  1.91671634e-19  1.43915343e-03
  -8.46560837e-04]
 [-1.68794411e-19 -8.28338878e-11 -2.72416278e-19 -8.46560837e-04
   1.43915344e-03]]
Det(FIM) =  1153564478115.3643
Eigen Values (FIM):
 [  49.99999999   50.00000098 1687.49999135  437.50000321  625.        ]
Eigen Vectors (FIM)
 [[-9.95133330e-01 -9.85375821e-02  2.39072176e-18 -3.43820647e-18
  -6.43479599e-19]
 [-9.85375821e-02  9.95133330e-01  1.07599822e-09  5.43417191e-09
   3.80503493e-09]
 [-3.74938942e-10  3.78651709e-09  2.77845024e-16  3.25064013e-16
  -1.00000000e+00

In [ ]:
##New Cell
#Create Meshgrid for X1 and X2 and evaluate Y
len_mesh_data = 20
x1 = np.linspace(np.min(Xexp[:,0]),np.max(Xexp[:,0]),len_mesh_data)
x2 = np.linspace(np.min(Xexp[:,1]),np.max(Xexp[:,1]),len_mesh_data)
X1, X2 = np.meshgrid(x1,x2)
X_mesh = np.meshgrid(x1,x2)
X_mesh_reshape = np.array(X_mesh).reshape(2,-1).T
# #Creates an array for Y that will be filled with the for loop

#Calculate y_sim
Y_nlr = model(nlr_theta, theta_ref, X_mesh_reshape, indeces_to_consider)
        
#Reshape to correct dimension (Is there an easier wat to do this?)        
param_names = ["X1", "X2"]
levels = [100]
idcs_to_plot = [0]
y_sim = np.array(Y).reshape(len_mesh_data,-1).T
y_nlr = np.array(Y_nlr).reshape(len_mesh_data,-1).T
z = np.array([y_sim, y_nlr])
z_titles = ["Y Sim", "Y Least-Squares"]
xbins = 5
ybins = 5
zbins = 900
title= None

param_info_dict = {"true":None, "min_sse":None, "names":param_names, "idcs":idcs_to_plot}
plotters.plot_nlr_heat_maps(X_mesh, z, z_titles, levels, param_info_dict, log_data = False)

In [ ]:
#Plot error
Y_nlr_exp = model(nlr_theta, theta_ref, Xexp, indeces_to_consider)
error = (Yexp - Y_nlr_exp)
print("SSE = ", np.sum(error**2))
plt.plot(Y_nlr_exp, error,"b.",markersize=20, label = "Error")

plt.legend(fontsize=10,bbox_to_anchor=(0, 1.05, 1, 0.2),borderaxespad=0, loc = "lower right")

plt.minorticks_on() # turn on minor ticks
plt.tick_params(direction="in",top=True, right=True) 
plt.tick_params(which="minor",direction="in",top=True, right=True)

plt.locator_params(axis='y', nbins=10)
plt.locator_params(axis='x', nbins=10)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.grid(False)

# plt.title("Residuals")
plt.xlabel('Predicted Potential', fontsize=16,fontweight='bold')
plt.ylabel('Residuals', fontsize=16,fontweight='bold')
plt.show()

In [ ]:
#Create heat map data
#Create list of heat map theta data
heat_map_data_dict = {}

#Create a linspace for the number of dimensions and define number of points
dim_theta = len(theta_true_names)
dim_list = np.linspace(0, dim_theta-1, dim_theta)

#Create a list of all combinations (without repeats e.g no (1,1), (2,2)) of dimensions of theta
mesh_combos = np.array(list(combinations(dim_list, 2)), dtype = int)
n_points = 20

#Meshgrid set always defined by n_points**2
theta_set = np.tile(np.array(theta_true), (n_points**2, 1))

#Set x_vals
norm_x_vals = Xexp

#Loop over all possible theta combinations of 2
for i in range(len(mesh_combos)):
    #Create a copy of the true values to change the mehsgrid valus on
    theta_set_copy = np.copy(theta_set)
    #Set the indeces of theta_set for evaluation as each row of mesh_combos
    idcs = mesh_combos[i]
    #define name of parameter set as tuple ("param_1,param_2")
    data_set_name = (theta_true_names[idcs[0]], theta_true_names[idcs[1]])

    #Create a meshgrid of values of the 2 selected values of theta and reshape to the correct shape
    #Assume that theta1 and theta2 have equal number of points on the meshgrid
    theta1 = np.linspace(lower[idcs[0]], upper[idcs[0]], n_points)
    theta2 = np.linspace(lower[idcs[1]], upper[idcs[1]], n_points)
    theta12_mesh = np.array(np.meshgrid(theta1, theta2))
    theta12_vals = np.array(theta12_mesh).T.reshape(-1,2)

    #Set initial values for evaluation (true values) to meshgrid values
    theta_set_copy[:,idcs] = theta12_vals
    
    #Append data set to dictionary with name
    heat_map_data_dict[data_set_name] = theta_set_copy
    
hm_data_keys = list(heat_map_data_dict.keys())
# print(heat_map_data_dict[hm_data_keys[0]].shape)

In [ ]:
#New Cell
log_data = False
# save_figure = True
save_figure = False


#Get Number of pairs
combos = list(combinations(dim_list, 2))
pairs = len((list(combinations(dim_list, 2))))

#For each pair
for pair in range(pairs):
    #Make a meshgrid for each parameter
    idcs_to_plot = [int(combos[pair][i]) for i in range(len(combos[pair]))]
    theta_data = heat_map_data_dict[hm_data_keys[pair]].reshape(n_points, n_points, -1).T
    theta_mesh = np.take(theta_data, list(combos[pair]), axis=0)
    
    sse_sim = sse_func(heat_map_data_dict[hm_data_keys[pair]], theta_ref, indeces_to_consider, Xexp, Yexp)
    plot_axis_names = theta_true_names[idcs_to_plot]
    
    title = "Heat Map Pair " + "-".join(map(str, plot_axis_names))
    title = None

    z = np.array([sse_sim])
    if log_data == True:
        z_titles = ["ln("+ r"$\mathbf{e(\theta)_{sim}}$" + ")"]
        z = np.log(z)
    else:
        z_titles = [r"$\mathbf{e(\theta)_{sim}}$" + ")"]
    
#     z_save_names = ["sse_sim", "sse_nlr"]
#     path_end = '-'.join(z_save_names) 
    levels = [100]

    param_info_dict = {"true":theta_true, "min_sse":nlr_theta, "names":plot_axis_names, "idcs":idcs_to_plot}
    plotters.plot_nlr_heat_maps(theta_mesh, z, z_titles, levels, param_info_dict, log_data = False)